# Regression using the hetero graphs

- Predicting income, property values, and rents using the sociodemographics and mobility networks.
- Three regressions: only sociodemographics, only mobility networks, and mobility + sociodemographics. LASSO is used when the input variable size is large.

TBD
- Make the normalization method consistent. Include NYC as another example.
- Automate the choice of lambda, now it is still a handcrafted process.
- Refine the lambda values to further improve the performance. I think the room is large.



In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import copy
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from sklearn.linear_model import LinearRegression

# regression tools
import statsmodels.api as sm
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler


Bad key text.latex.unicode in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key text.latex.preview in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 125 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 157 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso

In [3]:
# read files
with open("../../data/02_intermediate/socioecon_boston_miami_chicago_ct_shp_dic.pickle", 'rb') as f:
    socioecon_shp_dic = pickle.load(f)

# hetero activity graphs    
with open("../../data/03_processed/A_home_activity_three_cities_unweighted_dic.pickle", 'rb') as f:
    A_home_unweighted_dic = pickle.load(f)

with open("../../data/03_processed/A_home_activity_three_cities_weighted_dic.pickle", 'rb') as f:
    A_home_weighted_dic = pickle.load(f)


In [8]:
for col in socioecon_shp_dic['boston'].columns:
    print(col)

STATEFP
COUNTYFP
TRACTCE
NAME
NAMELSAD
MTFCC
FUNCSTAT
ALAND
AWATER
INTPTLAT
INTPTLON
geometry
pop_total_2018
sex_total_2018
sex_male_2018
sex_female_2018
age_median_2018
households_2018
race_total_2018
race_white_2018
race_black_2018
race_native_2018
race_asian_2018
inc_total_pop_2018
inc_no_pop_2018
inc_with_pop_2018
inc_pop_10k_2018
inc_pop_1k_15k_2018
inc_pop_15k_25k_2018
inc_pop_25k_35k_2018
inc_pop_35k_50k_2018
inc_pop_50k_65k_2018
inc_pop_65k_75k_2018
inc_pop_75k_2018
inc_median_ind_2018
travel_total_to_work_2018
travel_driving_to_work_2018
travel_pt_to_work_2018
travel_taxi_to_work_2018
travel_cycle_to_work_2018
travel_walk_to_work_2018
travel_work_from_home_2018
edu_total_pop_2018
bachelor_male_25_34_2018
master_phd_male_25_34_2018
bachelor_male_35_44_2018
master_phd_male_35_44_2018
bachelor_male_45_64_2018
master_phd_male_45_64_2018
bachelor_male_65_over_2018
master_phd_male_65_over_2018
bachelor_female_25_34_2018
master_phd_female_25_34_2018
bachelor_female_35_44_2018
master_

In [9]:
# initialize performance and coefficients
performance_dic = {}
coeff_dic = {}

# parameters
threshold = 1.0 # 1.0, 10.0
top_k = 15 # save the top 15 coefficients for the LASSO results.


In [96]:
# ###
# with open('../../data/05_model_outputs/lasso_performance.pickle', 'rb') as f:
#     performance_dic = pickle.load(f)
    
# with open('../../data/05_model_outputs/lasso_coefficients.pickle', 'rb') as f:
#     coeff_dic = pickle.load(f)


## Part 1. Working on the full data set

- Combine the three data sets first.
- Work on the combined data set.

In [102]:
### if we want to work on the AVERAGE effect!
df_socioecon_shp_dic = {}
for city in ['boston', 'miami', 'chicago']:
    print(city)
    df_socioecon_shp = socioecon_shp_dic[city]
    for activity in A_home_unweighted_dic[city][threshold].keys():
        activity_vector = pd.DataFrame(A_home_unweighted_dic[city][threshold][activity].sum(axis = 1), columns = [activity])
        df_socioecon_shp = df_socioecon_shp.merge(activity_vector, 
                                   left_index = True, 
                                   right_index = True, 
                                   how = 'left')
    df_socioecon_shp.fillna(0.0, inplace=True)
    df_socioecon_shp_dic[city] = df_socioecon_shp

df_socioecon_shp = pd.concat(df_socioecon_shp_dic, axis = 0)
df_socioecon_shp.fillna(0.0, inplace=True)
print(df_socioecon_shp)


boston
miami
chicago
                    STATEFP COUNTYFP TRACTCE     NAME              NAMELSAD  \
boston  25009201100      25      009  201100     2011     Census Tract 2011   
        25009202101      25      009  202101  2021.01  Census Tract 2021.01   
        25009202102      25      009  202102  2021.02  Census Tract 2021.02   
        25009202200      25      009  202200     2022     Census Tract 2022   
        25009203100      25      009  203100     2031     Census Tract 2031   
...                     ...      ...     ...      ...                   ...   
chicago 17197884005      17      197  884005  8840.05  Census Tract 8840.05   
        17197884006      17      197  884006  8840.06  Census Tract 8840.06   
        17197884101      17      197  884101  8841.01  Census Tract 8841.01   
        17197884103      17      197  884103  8841.03  Census Tract 8841.03   
        17197980100      17      197  980100     9801     Census Tract 9801   

                     MTFCC FUN

In [103]:
# set up
city = 'total' # abuse the term...
performance_dic[city] = {}
coeff_dic[city] = {}

In [104]:
# set up the outputs
output_var = 'inc_median_household_2018'
# output_var = 'property_value_median_2018' 
# output_var = 'rent_median_2018' 

# output_var = 'inc_per_capita_annual_growth' 
# output_var = 'employment_unemployed_ratio_2018' # useless.

# Create the dictionaries to save the results.
performance_dic[city][output_var] = {}
coeff_dic[city][output_var] = {}


#### 1.1 Only sociodemographics

In [106]:
# create the baseline regressions
var_list_1 = ['pop_total_2016',
              'pop_density_2016',
              'age_median_2016', 
              'household_size_avg_2016',
              'sex_male_ratio_2016', 
              'race_white_ratio_2016',
              'edu_higher_edu_ratio_2016',            
              'vehicle_per_capita_2016']

# var_list_1 = top_k_coefficients_sorted.index

# predict inc_per_capita_2018
var_2 = output_var

# scaler = StandardScaler()
# X = df_socioecon_shp[var_list_1]
# # X = sm.add_constant(X)
# X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
# y = scaler.fit_transform(df_socioecon_shp[var_2].values.reshape(-1, 1))

X = pd.DataFrame(normalize(X, axis = 0), columns = X.columns)
X = sm.add_constant(X)
y = normalize(df_socioecon_shp[var_2].values.reshape(-1, 1), axis = 0)

# y = df_socioecon_shp[var_2].values.reshape(-1, 1)


# train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

reg = LinearRegression().fit(X_train, y_train)
r2_train = np.round(reg.score(X_train, y_train), decimals = 3)
r2_test = np.round(reg.score(X_test, y_test), decimals = 3)
print("Training R2 is: ", r2_train)
print("Testing R2 is: ", r2_test)

mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

Training R2 is:  0.67
Testing R2 is:  0.683
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.676
Model:                            OLS   Adj. R-squared:                  0.675
Method:                 Least Squares   F-statistic:                     999.8
Date:                Wed, 27 Apr 2022   Prob (F-statistic):               0.00
Time:                        10:34:36   Log-Likelihood:                 15660.
No. Observations:                3840   AIC:                        -3.130e+04
Df Residuals:                    3831   BIC:                        -3.125e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

In [107]:
# save results
performance_dic[city][output_var]['baseline'] = pd.DataFrame([r2_train, r2_test, len(var_list_1)], 
                                                              index = ['training', 'testing', 'coef counts'],
                                                              columns = ['baseline'])


#### 1.2 Only activity-based mobility networks

In [108]:
# union of all the activities from the three cities
joint_activity = set(A_home_unweighted_dic['boston'][1.0].keys()).union(set(A_home_unweighted_dic['miami'][1.0].keys())).union(set(A_home_unweighted_dic['chicago'][1.0].keys()))
joint_activity_list = list(joint_activity)

In [115]:
var_list_1 = copy.copy(joint_activity_list)

# var_list_1 = list(A_home_unweighted_dic[city][threshold].keys())
# var_list_1 = [var + ' (home)' for var in var_list_1] + [var + ' (activity)' for var in var_list_1]
# var_list_1

# print(len(var_list_1))

# var_list_1 = ['pop_total_2016',
#               'pop_density_2016',
#               'age_median_2016', 
#               'household_size_avg_2016',
#               'sex_male_ratio_2016', 
#               'race_white_ratio_2016',
#               'edu_bachelor_ratio_2016',
#               'edu_master_ratio_2016',
#               'edu_master_ratio_2016', 
#               'vehicle_per_capita_2016']

# var_2_list = ['inc_per_capita_2018', 'property_value_median_2018']
var_2 = output_var

# scaler = StandardScaler()
# X = df_socioecon_shp[var_list_1]
# # X = sm.add_constant(X)
# X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
# y = scaler.fit_transform(df_socioecon_shp[var_2].values.reshape(-1, 1))

X = df_socioecon_shp[var_list_1]
X = pd.DataFrame(normalize(X, axis = 0), columns = X.columns)
X = sm.add_constant(X)
y = normalize(df_socioecon_shp[var_2].values.reshape(-1, 1), axis = 0)

# y = df_socioecon_shp[var_2].values.reshape(-1, 1)


# split training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

# searching in L1 regularized regressions
alpha_list = [0.00, 0.000001, 0.000002, 0.000005, 0.00001, 0.00002, 0.00003, 0.00004, 0.00005, 0.0001, 0.001, 0.01, 0.1, 1.0]
# alpha_list = [0.00, 0.01, 0.1, 1.0, 5.0, 10.0, 20.0, 50.0, 100, 200]
# alpha_list = [0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.08, 0.1, 0.5, 1.0] # for growth data


performance_table = pd.DataFrame(np.zeros((3, len(alpha_list))),
                                 index = ['training', 'testing', 'coef counts'],
                                 columns = alpha_list)

for alpha in alpha_list:
    model = Lasso(alpha = alpha)
    model.fit(X_train, y_train)    
    train_r2 = model.score(X_train, y_train)
    test_r2 = model.score(X_test, y_test)
    # count the non-zero coef
    pos_counts = np.sum(model.coef_ > 0.01)
    neg_counts = np.sum(model.coef_ < -0.01)
    total_counts = pos_counts + neg_counts
    
    performance_table.loc['training', alpha] = train_r2
    performance_table.loc['testing', alpha] = test_r2
    performance_table.loc['coef counts', alpha] = int(total_counts)

/home/jtl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
/home/jtl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/home/jtl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.928e-02, tolerance: 1.332e-05 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  coef_, l1_reg, l2_reg, X, y, max_i

In [116]:
performance_table

,0.000000,0.000001,0.000002,0.000005,0.000010,0.000020,0.000030,0.000040,0.000050,0.000100,0.001000,0.010000,0.100000,1.000000
training,0.710500,0.633060,0.583286,0.484645,0.383029,0.199699,0.06260,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
testing,0.508544,0.575116,0.557957,0.491665,0.399363,0.215641,0.07027,-0.000232,-0.000232,-0.000232,-0.000232,-0.000232,-0.000232,-0.000232
coef counts,291.000000,78.000000,48.000000,25.000000,10.000000,5.000000,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


 highest performance is about 59% R2. Using only 15 activities, we could get about 43% R2.

In [117]:
# Rerun the model to find the important coefficients
alpha = 0.000005
# alpha = 50.0
model = Lasso(alpha = alpha)
model.fit(X_train, y_train)   

# col names 
full_col_names = ['const'] + var_list_1
# full_col_names = var_list_1

# combine coefficients and names
coef = pd.DataFrame(model.coef_, index = full_col_names, columns = ['value'])
pd.set_option('display.max_rows', 500)
coef.sort_values('value')

,value
Discount Store,-0.149846
Laundromat,-0.028387
Caribbean,-0.025990
Latin American,-0.025164
Storage,-0.017319
...,...
French,0.027658
Football,0.034704
Hockey,0.076350
New American,0.093679


In [118]:
# save performance and coefficients
performance_dic[city][output_var]['lasso (no socio-demographics)'] = performance_table

# save coefficients
top_k_index = np.abs(coef).sort_values('value').iloc[-top_k:,:].index
top_k_coefficients = coef.loc[top_k_index, :]
top_k_coefficients_sorted = top_k_coefficients.sort_values('value')
coeff_dic[city][output_var]['lasso (no socio-demographics)'] = top_k_coefficients_sorted


In [119]:
top_k_coefficients_sorted

,value
Discount Store,-0.149846
Laundromat,-0.028387
Caribbean,-0.025990
Latin American,-0.025164
Gastropub,0.018128
Wine Shop,0.019050
Golf Course,0.019757
Stadium,0.020286
Cycle Studio,0.022244
Tennis Court,0.023309


#### 1.3 Combining the activity networks and sociodemographics

In [122]:
var_list_1 = ['pop_total_2016',
              'pop_density_2016',
              'age_median_2016', 
              'household_size_avg_2016',
              'sex_male_ratio_2016', 
              'race_white_ratio_2016',
              'edu_higher_edu_ratio_2016',            
               'vehicle_per_capita_2016']

# var_list_1 += list(A_home_unweighted_dic[city][threshold].keys())
var_list_1 += joint_activity_list

# print(len(var_list_1))

# var_2_list = ['inc_per_capita_2018', 'property_value_median_2018']
var_2 = output_var

# scaler = StandardScaler()
# X = df_socioecon_shp[var_list_1]
# # X = sm.add_constant(X)
# X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
# y = scaler.fit_transform(df_socioecon_shp[var_2].values.reshape(-1, 1))

X = df_socioecon_shp[var_list_1]
X = sm.add_constant(X)
X = pd.DataFrame(normalize(X, axis = 0), columns = X.columns)
# y = df_socioecon_shp[var_2].values
y = normalize(df_socioecon_shp[var_2].values.reshape(-1, 1), axis = 0)


# split training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

# searching in L1 regularized regressions
alpha_list = [0.00, 0.000001, 0.000002, 0.000005, 0.00001, 0.00002, 0.00003, 0.00004, 0.00005, 0.0001, 0.001, 0.01, 0.1, 1.0]
# alpha_list = [0.00, 0.001,0.005,0.008, 0.01, 0.02, 0.05, 0.1, 1.0]
# alpha_list = [0.00, 0.1, 1.0, 5.0, 10.0, 20.0, 50.0, 100, 200]

performance_table = pd.DataFrame(np.zeros((3, len(alpha_list))),
                                 index = ['training', 'testing', 'coef counts'],
                                 columns = alpha_list)

for alpha in alpha_list:
    model = Lasso(alpha = alpha)
    model.fit(X_train, y_train)
    train_r2 = model.score(X_train, y_train)
    test_r2 = model.score(X_test, y_test)
    # count the non-zero coef
    pos_counts = np.sum(model.coef_ > 0.01)
    neg_counts = np.sum(model.coef_ < -0.01)
    total_counts = pos_counts + neg_counts
    
    performance_table.loc['training', alpha] = train_r2
    performance_table.loc['testing', alpha] = test_r2
    performance_table.loc['coef counts', alpha] = int(total_counts)

/home/jtl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
/home/jtl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/home/jtl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.254e-02, tolerance: 1.332e-05 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  coef_, l1_reg, l2_reg, X, y, max_i

In [123]:
performance_table

,0.000000,0.000001,0.000002,0.000005,0.000010,0.000020,0.000030,0.000040,0.000050,0.000100,0.001000,0.010000,0.100000,1.000000
training,0.811708,0.744149,0.708424,0.657422,0.601221,0.459091,0.277804,0.055902,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
testing,0.673506,0.723889,0.712207,0.684323,0.629727,0.479082,0.287651,0.057272,-0.000232,-0.000232,-0.000232,-0.000232,-0.000232,-0.000232
coef counts,238.000000,33.000000,18.000000,7.000000,4.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


jointly we get 73% R2. Using only nine variables, the highest R2 is 70%. Sociodemographics, particularly education and population density are important signals.

In [124]:
# Rerun the model to find the important coefficients
# alpha = 50.0
alpha = 0.000002
model = Lasso(alpha = alpha)
model.fit(X_train, y_train)   

# col names 
full_col_names = ['const'] + var_list_1
# full_col_names = var_list_1

# combine coefficients and names
coef = pd.DataFrame(model.coef_, index = full_col_names, columns = ['value'])
pd.set_option('display.max_rows', 700)
coef.sort_values('value')


,value
pop_density_2016,-0.087317
household_size_avg_2016,-0.021304
Gay Bar,-0.019271
Beach,-0.014757
Big Box Store,-0.014660
Communications,-0.011227
Hookah Bar,-0.008064
Casino,-0.008044
Pakistani,-0.007859
Indian,-0.007780


In [125]:
# save performance and coefficients
performance_dic[city][output_var]['lasso (with socio-demographics)'] = performance_table

# save coefficients
top_k_index = np.abs(coef).sort_values('value').iloc[-top_k:,:].index
top_k_coefficients = coef.loc[top_k_index, :]
top_k_coefficients_sorted = top_k_coefficients.sort_values('value')
coeff_dic[city][output_var]['lasso (with socio-demographics)'] = top_k_coefficients_sorted


In [126]:
top_k_coefficients_sorted

,value
pop_density_2016,-0.087317
household_size_avg_2016,-0.021304
Gay Bar,-0.019271
Beach,-0.014757
Big Box Store,-0.014660
Baseball,0.012157
New American,0.014263
Cycle Studio,0.014271
Tennis Court,0.014814
Sports Club,0.015676


# Part 2. Working on the three cities separately.

- I ran the scripts nine times (3 * 3) by HANDS! This process needs to be automated!
- Three cities
- Three output variables

In [131]:
# Set up parameters
city = 'boston'
# city = 'chicago'
# city = 'miami'

# 
performance_dic[city] = {}
coeff_dic[city] = {}

In [132]:
# working on individual cities
# augment the activity data to sociodemographics
df_socioecon_shp = socioecon_shp_dic[city]

for activity in A_home_unweighted_dic[city][threshold].keys():
    activity_vector = pd.DataFrame(A_home_unweighted_dic[city][threshold][activity].sum(axis = 1), columns = [activity])
    df_socioecon_shp = df_socioecon_shp.merge(activity_vector, 
                               left_index = True, 
                               right_index = True, 
                               how = 'left')

df_socioecon_shp.fillna(0.0, inplace=True)
df_socioecon_shp


,STATEFP,COUNTYFP,TRACTCE,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,...,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings,Women's Store,Yoga Studio,Yogurt,Zoo,Zoo Exhibit
25009201100,25,009,201100,2011,Census Tract 2011,G5020,S,2786306,3481257,+42.4282342,...,0.0,3.0,3.0,1.0,1.0,5.0,5.0,2.0,0.0,0.0
25009202101,25,009,202101,2021.01,Census Tract 2021.01,G5020,S,1454942,7226,+42.4758943,...,0.0,3.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
25009202102,25,009,202102,2021.02,Census Tract 2021.02,G5020,S,2919051,24048,+42.4829911,...,0.0,2.0,0.0,1.0,1.0,6.0,1.0,2.0,0.0,0.0
25009202200,25,009,202200,2022,Census Tract 2022,G5020,S,3452798,1441782,+42.4695768,...,1.0,3.0,0.0,1.0,2.0,7.0,3.0,5.0,0.0,0.0
25009203100,25,009,203100,2031,Census Tract 2031,G5020,S,3720285,1481336,+42.4871332,...,1.0,6.0,4.0,0.0,1.0,4.0,5.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33017085000,33,017,085000,850,Census Tract 850,G5020,S,120890933,5035519,+43.2155657,...,0.0,0.0,0.0,1.0,3.0,5.0,4.0,2.0,0.0,0.0
33017086000,33,017,086000,860,Census Tract 860,G5020,S,126932596,5722104,+43.2753082,...,1.0,0.0,3.0,1.0,2.0,3.0,4.0,3.0,0.0,0.0
33017087000,33,017,087000,870,Census Tract 870,G5020,S,94758837,909784,+43.3669235,...,0.0,1.0,1.0,1.0,2.0,3.0,0.0,3.0,0.0,0.0
33017088000,33,017,088000,880,Census Tract 880,G5020,S,85593614,3177277,+43.4522291,...,0.0,2.0,1.0,1.0,3.0,5.0,2.0,3.0,0.0,0.0


In [133]:
# output variables
output_var = 'inc_median_household_2018'
# output_var = 'property_value_median_2018' 
# output_var = 'rent_median_2018' 

# output_var = 'inc_per_capita_2018' 
# output_var = 'employment_unemployed_ratio_2018' # useless.

# Create the dictionaries to save the results.
performance_dic[city][output_var] = {}
coeff_dic[city][output_var] = {}


#### 2.1 only sociodemographics

In [138]:
# create the baseline regressions
var_list_1 = ['pop_total_2016',
              'pop_density_2016',
              'age_median_2016', 
              'household_size_avg_2016',
              'sex_male_ratio_2016', 
              'race_white_ratio_2016',
              'edu_higher_edu_ratio_2016',            
              'vehicle_per_capita_2016']

# predict inc_per_capita_2018
var_2 = output_var

# scaler = StandardScaler()
# X = df_socioecon_shp[var_list_1]
# # X = sm.add_constant(X)
# X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
# y = scaler.fit_transform(df_socioecon_shp[var_2].values.reshape(-1, 1), )

X = df_socioecon_shp[var_list_1]
X = pd.DataFrame(normalize(X, axis = 0), columns = X.columns)
X = sm.add_constant(X)
y = normalize(df_socioecon_shp[var_2].values.reshape(-1, 1), axis = 0)

# y = df_socioecon_shp[var_2].values.reshape(-1, 1)


# train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

reg = LinearRegression().fit(X_train, y_train)
r2_train = np.round(reg.score(X_train, y_train), decimals = 3)
r2_test = np.round(reg.score(X_test, y_test), decimals = 3)
print("Training R2 is: ", r2_train)
print("Testing R2 is: ", r2_test)

mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

Training R2 is:  0.688
Testing R2 is:  0.558
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.656
Model:                            OLS   Adj. R-squared:                  0.653
Method:                 Least Squares   F-statistic:                     234.5
Date:                Wed, 27 Apr 2022   Prob (F-statistic):          6.95e-222
Time:                        10:39:59   Log-Likelihood:                 3496.9
No. Observations:                 992   AIC:                            -6976.
Df Residuals:                     983   BIC:                            -6932.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

In [139]:
# save results
performance_dic[city][output_var]['baseline'] = pd.DataFrame([r2_train, r2_test, len(var_list_1)], 
                                                              index = ['training', 'testing', 'coef counts'],
                                                              columns = ['baseline'])


#### 2.2 only activities

In [140]:
for col in df_socioecon_shp.columns:
    print(col)

STATEFP
COUNTYFP
TRACTCE
NAME
NAMELSAD
MTFCC
FUNCSTAT
ALAND
AWATER
INTPTLAT
INTPTLON
geometry
pop_total_2018
sex_total_2018
sex_male_2018
sex_female_2018
age_median_2018
households_2018
race_total_2018
race_white_2018
race_black_2018
race_native_2018
race_asian_2018
inc_total_pop_2018
inc_no_pop_2018
inc_with_pop_2018
inc_pop_10k_2018
inc_pop_1k_15k_2018
inc_pop_15k_25k_2018
inc_pop_25k_35k_2018
inc_pop_35k_50k_2018
inc_pop_50k_65k_2018
inc_pop_65k_75k_2018
inc_pop_75k_2018
inc_median_ind_2018
travel_total_to_work_2018
travel_driving_to_work_2018
travel_pt_to_work_2018
travel_taxi_to_work_2018
travel_cycle_to_work_2018
travel_walk_to_work_2018
travel_work_from_home_2018
edu_total_pop_2018
bachelor_male_25_34_2018
master_phd_male_25_34_2018
bachelor_male_35_44_2018
master_phd_male_35_44_2018
bachelor_male_45_64_2018
master_phd_male_45_64_2018
bachelor_male_65_over_2018
master_phd_male_65_over_2018
bachelor_female_25_34_2018
master_phd_female_25_34_2018
bachelor_female_35_44_2018
master_

In [141]:
# assigne inputs and outputs
var_list_1 = list(A_home_unweighted_dic[city][threshold].keys())
# var_list_1 = joint_activity_list

# var_list_1 = list(A_home_unweighted_dic[city][threshold].keys())
# var_list_1 = [var + ' (home)' for var in var_list_1] + [var + ' (activity)' for var in var_list_1]
# var_list_1

# print(len(var_list_1))

# var_list_1 = ['pop_total_2016',
#               'pop_density_2016',
#               'age_median_2016', 
#               'household_size_avg_2016',
#               'sex_male_ratio_2016', 
#               'race_white_ratio_2016',
#               'edu_bachelor_ratio_2016',
#               'edu_master_ratio_2016',
#               'edu_master_ratio_2016', 
#               'vehicle_per_capita_2016']

# var_2_list = ['inc_per_capita_2018', 'property_value_median_2018']
var_2 = output_var

# scaler = StandardScaler()
# X = df_socioecon_shp[var_list_1]
# # X = sm.add_constant(X)
# X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
# y = scaler.fit_transform(df_socioecon_shp[var_2].values.reshape(-1, 1))

X = df_socioecon_shp[var_list_1]
X = sm.add_constant(X)
X = pd.DataFrame(normalize(X, axis = 0), columns = X.columns)
y = normalize(df_socioecon_shp[var_2].values.reshape(-1, 1), axis = 0)

# y = df_socioecon_shp[var_2].values.reshape(-1, 1)


# split training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

# searching in L1 regularized regressions
alpha_list = [0.00, 0.000001, 0.000002, 0.000005, 0.00001, 0.00002, 0.00003, 0.00004, 0.00005, 0.0001, 0.001, 0.01, 0.1, 1.0]
# alpha_list = [0.00, 0.01, 0.1, 1.0, 5.0, 10.0, 20.0, 50.0, 100, 200]

performance_table = pd.DataFrame(np.zeros((3, len(alpha_list))),
                                 index = ['training', 'testing', 'coef counts'],
                                 columns = alpha_list)

for alpha in alpha_list:
    model = Lasso(alpha = alpha)
    model.fit(X_train, y_train)    
    train_r2 = model.score(X_train, y_train)
    test_r2 = model.score(X_test, y_test)
    # count the non-zero coef
    pos_counts = np.sum(model.coef_ > 0.01)
    neg_counts = np.sum(model.coef_ < -0.01)
    total_counts = pos_counts + neg_counts
    
    performance_table.loc['training', alpha] = train_r2
    performance_table.loc['testing', alpha] = test_r2
    performance_table.loc['coef counts', alpha] = int(total_counts)

/home/jtl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
/home/jtl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/home/jtl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.747e-04, tolerance: 1.042e-05 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  coef_, l1_reg, l2_reg, X, y, max_i

In [142]:
performance_table

,0.000000,0.000001,0.000002,0.000005,0.000010,0.000020,0.000030,0.000040,0.000050,0.000100,0.001000,0.010000,0.100000,1.000000
training,0.983203,0.910956,0.867143,0.773222,0.678802,0.566936,0.482394,0.433300,0.381129,0.094770,0.00000,0.00000,0.00000,0.00000
testing,-10.114372,0.087613,0.322553,0.469298,0.472405,0.445420,0.399938,0.361868,0.318637,0.074366,-0.00048,-0.00048,-0.00048,-0.00048
coef counts,566.000000,257.000000,172.000000,82.000000,53.000000,23.000000,16.000000,12.000000,8.000000,4.000000,0.00000,0.00000,0.00000,0.00000


In [143]:
# Rerun the model to find the important coefficients
alpha = 0.00002
# alpha = 50.0
model = Lasso(alpha = alpha)
model.fit(X_train, y_train)   

# col names 
full_col_names = ['const'] + var_list_1
# full_col_names = var_list_1


# combine coefficients and names
coef = pd.DataFrame(model.coef_, index = full_col_names, columns = ['value'])
pd.set_option('display.max_rows', 500)
coef.sort_values('value')

,value
Latin American,-0.057814
Caribbean,-0.055141
Nightclub,-0.020932
Billiards,-0.019687
Laundromat,-0.016612
...,...
Cupcakes,0.038055
Hockey,0.039511
New American,0.054624
Skating Rink,0.058190


In [144]:
# save performance and coefficients
performance_dic[city][output_var]['lasso (no socio-demographics)'] = performance_table

# save coefficients
top_k_index = np.abs(coef).sort_values('value').iloc[-top_k:,:].index
top_k_coefficients = coef.loc[top_k_index, :]
top_k_coefficients_sorted = top_k_coefficients.sort_values('value')
coeff_dic[city][output_var]['lasso (no socio-demographics)'] = top_k_coefficients_sorted


In [145]:
top_k_coefficients_sorted

,value
Latin American,-0.057814
Caribbean,-0.055141
Nightclub,-0.020932
Billiards,-0.019687
Food Stand,0.019499
Science Museum,0.019783
Football,0.021699
Ski Area,0.025363
Peking Duck,0.030332
Soccer,0.030729


#### 2.3 combining activities and sociodemographics

In [146]:
var_list_1 = ['pop_total_2016',
              'pop_density_2016',
              'age_median_2016', 
              'household_size_avg_2016',
              'sex_male_ratio_2016', 
              'race_white_ratio_2016',
              'edu_higher_edu_ratio_2016',            
               'vehicle_per_capita_2016']

var_list_1 += list(A_home_unweighted_dic[city][threshold].keys())
# var_list_1 += joint_activity_list

# print(len(var_list_1))

# var_2_list = ['inc_per_capita_2018', 'property_value_median_2018']
var_2 = output_var

# scaler = StandardScaler()
# X = df_socioecon_shp[var_list_1]
# # X = sm.add_constant(X)
# X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
# y = scaler.fit_transform(df_socioecon_shp[var_2].values.reshape(-1, 1))

X = df_socioecon_shp[var_list_1]
X = sm.add_constant(X)
X = pd.DataFrame(normalize(X, axis = 0), columns = X.columns)
# y = df_socioecon_shp[var_2].values
y = normalize(df_socioecon_shp[var_2].values.reshape(-1, 1), axis = 0)


# split training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

# searching in L1 regularized regressions
alpha_list = [0.00, 0.000001, 0.000002, 0.000005, 0.00001, 0.00002, 0.00005, 0.01, 0.1, 1.0]
# alpha_list = [0.00, 0.1, 1.0, 5.0, 10.0, 20.0, 50.0, 100, 200]

performance_table = pd.DataFrame(np.zeros((3, len(alpha_list))),
                                 index = ['training', 'testing', 'coef counts'],
                                 columns = alpha_list)

for alpha in alpha_list:
    model = Lasso(alpha = alpha)
    model.fit(X_train, y_train)
    train_r2 = model.score(X_train, y_train)
    test_r2 = model.score(X_test, y_test)
    # count the non-zero coef
    pos_counts = np.sum(model.coef_ > 0.01)
    neg_counts = np.sum(model.coef_ < -0.01)
    total_counts = pos_counts + neg_counts
    
    performance_table.loc['training', alpha] = train_r2
    performance_table.loc['testing', alpha] = test_r2
    performance_table.loc['coef counts', alpha] = int(total_counts)

/home/jtl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
/home/jtl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/home/jtl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.758e-04, tolerance: 1.042e-05 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  coef_, l1_reg, l2_reg, X, y, max_i

In [147]:
performance_table

,0.000000,0.000001,0.000002,0.000005,0.000010,0.000020,0.000050,0.010000,0.100000,1.000000
training,0.990863,0.934044,0.899368,0.828315,0.765546,0.690732,0.565194,0.00000,0.00000,0.00000
testing,-4.820535,0.455415,0.565231,0.619584,0.608331,0.577686,0.487120,-0.00048,-0.00048,-0.00048
coef counts,526.000000,185.000000,116.000000,49.000000,24.000000,10.000000,7.000000,0.00000,0.00000,0.00000


In [148]:
# Rerun the model to find the important coefficients
# alpha = 50.0
alpha = 0.00001
model = Lasso(alpha = alpha)
model.fit(X_train, y_train)   

# col names 
full_col_names = ['const'] + var_list_1
# full_col_names = var_list_1

# combine coefficients and names
coef = pd.DataFrame(model.coef_, index = full_col_names, columns = ['value'])
pd.set_option('display.max_rows', 700)
coef.sort_values('value')


,value
pop_density_2016,-0.099429
Billiards,-0.021653
Waterfall,-0.016318
Nightclub,-0.016110
Communications,-0.014980
Apres Ski Bar,-0.014099
Shrine,-0.013913
Preserve,-0.012738
Waterfront,-0.011305
Bus Station,-0.010821


In [149]:
# save performance and coefficients
performance_dic[city][output_var]['lasso (with socio-demographics)'] = performance_table

# save coefficients
top_k_index = np.abs(coef).sort_values('value').iloc[-top_k:,:].index
top_k_coefficients = coef.loc[top_k_index, :]
top_k_coefficients_sorted = top_k_coefficients.sort_values('value')
coeff_dic[city][output_var]['lasso (with socio-demographics)'] = top_k_coefficients_sorted


In [150]:
top_k_coefficients_sorted

,value
pop_density_2016,-0.099429
Billiards,-0.021653
Waterfall,-0.016318
Nightclub,-0.016110
Communications,-0.014980
Fishing Store,0.014875
Peking Duck,0.015592
Hockey,0.018001
Ski Area,0.023624
Food Stand,0.026291


# Save

- Don't save until the 3 + 3 * 3 = 12 cases are all exhausted.

In [51]:
# ###
# with open('../../data/05_model_outputs/lasso_performance.pickle', 'wb') as f:
#     pickle.dump(performance_dic, f)
    
# with open('../../data/05_model_outputs/lasso_coefficients.pickle', 'wb') as f:
#     pickle.dump(coeff_dic, f)
